In [1]:
import os
from typing import Any, List

from dotenv import load_dotenv
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import Tool, StructuredTool
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field

from db.db import get_session
from db.models import Grade
from service.service import BaseService

In [2]:
load_dotenv()
LITE_LLM_API_KEY = os.getenv("OPENAI_API_KEY")


model = ChatOpenAI(
    model="gpt-4o", api_key=LITE_LLM_API_KEY, streaming=True, verbose=True
)


class GuidanceHelperStdOutput(BaseModel):
    has_user_answered: bool = Field(
        description="Whether the user has correctly answered the topic at hand"
    )
    expertise_level: str = Field(
        description="The expertise user has self evaluated himself with"
    )
    expertise_id: int = Field(description="The expertise or grade ID")
    message: str = Field(description="Message to send to the user")


In [3]:
async def get_grades_or_expertise() -> List[Grade]:
    """
    Useful tool to retrieve current grades or expertise level grading system
    :return: List of json representing those grades and all their fields
    """
    async for session in get_session():
        service: BaseService[Grade, int, Any, Any] = BaseService(Grade, session)
        all_db_grades = await service.list_all()
        all_grades_json: List[str] = []
        for grade in all_db_grades:
            json_grade = grade.model_dump_json()
            all_grades_json.append(json_grade)
        return all_grades_json


async def get_expertises_or_skills() -> List[Grade]:
    """
    Useful tool to retrieve current grades or expertise level grading system
    :return: List of json representing those grades and all their fields
    """
    async for session in get_session():
        service: BaseService[Grade, int, Any, Any] = BaseService(Grade, session)
        all_db_grades = await service.list_all()
        all_grades_json: List[str] = []
        for grade in all_db_grades:
            json_grade = grade.model_dump_json()
            all_grades_json.append(json_grade)
        return all_grades_json


In [4]:
from dto.response.matrix_chats import MessageDict

msgs: List[MessageDict] = [
    MessageDict(
        msg_type="ai",
        message="""
        We're excited to have you here to discuss the crucial topic of Docker Networking. Our discussion aims to help you understand and select the appropriate expertise level for this skill. It's essential to align your understanding and capabilities with the right expertise level that suits your needs.

The available expertise levels range from basic awareness to expert-level proficiency:

Not Informed: For individuals new to the concept.
Informed Basics: For those with a foundational understanding.
Informed in Details: For users with a detailed grasp of concepts.
Practice and Lab Examples: For hands-on experience with Docker networking scenarios.
Production Maintenance: For maintaining Docker environments.
Production from Scratch: For building Docker-based solutions from the ground up.
Educator/Expert: For those who can teach or provide expert guidance on Docker networking.
Evaluate your current knowledge and the requirements of your role to determine which level best applies to you. This way, you'll ensure you're equipped with the right level of expertise to tackle Docker networking challenges effectively. Let's dive in and explore the best fit for your skills and aspirations!
        """
    ),
    MessageDict(
        msg_type="human",
        message="""
        What is the difference between:
        Not Informed: For individuals new to the concept.
        Informed Basics: For those with a foundational understanding.
        """
    ),
    # MessageDict(
    #     msg_type="ai",
    #     message="""
    #     Please select an expertise level that best represents your current understanding of Cryptography. This will help us identify where your strengths lie and areas for improvement:\n\n- Not Informed\n- Informed Basics\n- Informed in Details\n- Practice and Lab Examples\n- Production Maintenance\n- Production from Scratch\n- Educator/Expert
    #     """
    # ),
    # MessageDict(
    #     msg_type="human",
    #     message="""
    #     Working great
    #     """
    # ),
    # MessageDict(
    #     msg_type="ai",
    #     message="""
    #     Please select an expertise level that best represents your current understanding of Cryptography. This will help us identify where your strengths lie and areas for improvement:\n\n- Not Informed\n- Informed Basics\n- Informed in Details\n- Practice and Lab Examples\n- Production Maintenance\n- Production from Scratch\n- Educator/Expert
    #     """
    # ),
    # MessageDict(
    #     msg_type="human",
    #     message="""
    #     What is the difference between informed in basics and informed in details?
    #     """
    # ),
    # MessageDict(
    #     msg_type="ai",
    #     message="""
    #     The difference between "Informed Basics" and "Informed in Details" in the context of expertise in Cryptography can be understood as follows:\n\n- **Informed Basics**: This level indicates familiarity with fundamental concepts of cryptography. You have a basic understanding and can perform elementary tasks or recognize key concepts, but may not possess a deep comprehension of the subject.\n\n- **Informed in Details**: At this level, you have comprehensive knowledge of cryptography. You understand complex ideas, can handle intricate issues, and demonstrate a thorough grasp of the subject. This level reflects a more sophisticated and in-depth understanding than "Informed Basics."\n\nWould you like to select an expertise level now?
    #     """
    # ),
    # MessageDict(
    #     msg_type="human",
    #     message="""
    #     What is the topic at hand?
    #     """
    # ),
    # MessageDict(
    #     msg_type="ai",
    #     message="""
    #     I'm here to assist you with evaluating your expertise level in Cryptography. Please select an expertise level that best represents your current understanding. If you have questions about the differences between the levels, feel free to ask!
    #     """
    # ),
    # MessageDict(
    #     msg_type="human",
    #     message="""
    #     Informed in Details
    #     """
    # ),
#     MessageDict(
#         msg_type="ai",
#         message="""
#         User,
#
#         I noticed that the answers you've been providing are not directly addressing the questions asked and seem to be off-topic or unrelated. It's important for us to maintain clear and relevant communication to ensure that we can assist you effectively.
#
#         If this pattern continues, we may need to escalate the issue to our managers for further review. Please let us know if there's anything we can do to help or clarify things for you.
#
#         Thank you for your understanding and cooperation.
#         """
#     ),
#     MessageDict(
#         msg_type="human",
#         message="""
#         Practice and Lab Examples
#         """
#     ),
]

In [5]:
from utils.common import convert_msg_dict_to_langgraph_format

tools = [StructuredTool.from_function(
    function=get_grades_or_expertise,
    coroutine=get_grades_or_expertise,
)]
system_msg = """
You are helping the user to properly grade their expertise in the mentioned field.
Everything you help him with should be done by utilizing the tools or around the topic
of helping him populate his expertise level on the topic.
Do not discuss anything except from the provided context.
You are guiding the user to evaluate himself on provided topic.
Do not discuss anything (any other topic) except from the ones provided in topic!
Do not chat about other topics with the user, guide him how to populate his expertise with the grades provided
Warn the user if answering with unrelated topics or evading to answer the question will be escalated by involving managers!
Topic: {context}
If the user is evading to answer the question and is not asking any questions related to the topic for 4 or 5 messages
please respond with exactly:"ASKING_ADMIN"
When the user answers with proper categorization of skills return only that categorization!
"""
agent = create_react_agent(model=model, tools=tools, response_format=GuidanceHelperStdOutput)
messages = convert_msg_dict_to_langgraph_format(msgs)
async for chunk in agent.astream({"messages": [SystemMessage(system_msg)] + messages, "context": messages[0]}):
    print("RESPONSE ->>>>>>>>>>>>", chunk)

RESPONSE ->>>>>>>>>>>> {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4672pzsaGAv7aBXoEdrA0Nlt', 'function': {'arguments': '{}', 'name': 'get_grades_or_expertise'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'service_tier': 'default'}, id='run--2ce3c460-0e0d-4c41-a6e0-047a7bd5ec21-0', tool_calls=[{'name': 'get_grades_or_expertise', 'args': {}, 'id': 'call_4672pzsaGAv7aBXoEdrA0Nlt', 'type': 'tool_call'}])]}}
RESPONSE ->>>>>>>>>>>> {'tools': {'messages': [ToolMessage(content='["{\\"value\\":1,\\"id\\":1,\\"deleted\\":false,\\"label\\":\\"Not Informed\\"}", "{\\"value\\":2,\\"id\\":2,\\"deleted\\":false,\\"label\\":\\"Informed Basics\\"}", "{\\"value\\":3,\\"id\\":4,\\"deleted\\":false,\\"label\\":\\"Informed in details\\"}", "{\\"value\\":4,\\"id\\":5,\\"deleted\\":false,\\"label\\":\\"Practice and lab examples\\"}", "{\\"value\